In [17]:
import requests
from collections import namedtuple
# import validators
from bs4 import BeautifulSoup, NavigableString
from halo import Halo
def get_soup(url):
    '''
    Returns a BeautifulSoup version of a web page.
    '''
    response = requests.get(url)
    content = response.text
    return BeautifulSoup(content)

def get_links(table):
    '''
    Retrieves links from an embedded tabular structure.
    '''
    links = set()
    for child in table.children:
        for link in child.find_all('a', href=True):
            links.add(link['href'])

    return links

In [50]:
url = 'http://christmas-specials.wikia.com/'
    #data='xmas'
    # if not validators.url(url):
    #     print('Invalid URL')
    # path = pathlib.Path(data)

spinner = Halo(text='Fetching article list . . .')
spinner.start()
    
soup = get_soup(url + '/wiki/Special:AllPages')
spinner.succeed(text='Fetching list ===> Done')

'''Process the articles'''

articles = set()

spinner = Halo(text='Processing articles . . .')
spinner.start()

table_chunks = soup.find('table', 
        {'class': ['allpageslist', 'mw-allpages-table-chunk']})
    
chunks = set(get_links(table_chunks))

chunks_done = 0
chunks_left = len(chunks)

⠋ Fetching article list . . .⠙ Fetching article list . . .⠹ Fetching article list . . .⠸ Fetching article list . . .⠼ Fetching article list . . .⠴ Fetching article list . . .✔ Fetching list ===> Done
⠙ Processing articles . . .

In [70]:
article = '/wiki/Mr._Monk_and_the_Man_Who_Shot_Santa_Claus'
def get_article_data(article, url):
    article = namedtuple('Article', 'title contents categories')
    article_soup = get_soup(url + article)
    article_contents = article_soup.find_all('div', {'class': ['mw-content-ltr', 'mw-content-text', 'mw-collapsible', 'mw-made-collapsible']})
    article_categories = article_soup.find_all('nav', {'id': 'articleCategories'})


In [71]:
article_contents

[<nav class="article-categories CategorySelect articlePage" id="articleCategories">
 <div class="container">
 <div class="special-categories"><a class="categoriesLink" href="/wiki/Special:Categories" rel="nofollow" title="Special:Categories">Categories</a>:</div>
 <ul class="categories">
 <li class="category normal" data-name="Episodes" data-namespace="" data-outertag="" data-sortkey="" data-type="normal">
 <span class="name"><a href="/wiki/Category:Episodes" title="Category:Episodes">Episodes</a></span>
 <ul class="toolbar">
 <li class="tool editCategory sprite-small edit" title=""></li>
 <li class="tool removeCategory sprite-small delete" title=""></li>
 </ul>
 </li>
 <li class="category normal" data-name="Originally aired on the USA Network" data-namespace="" data-outertag="" data-sortkey="" data-type="normal">
 <span class="name"><a href="/wiki/Category:Originally_aired_on_the_USA_Network" title="Category:Originally aired on the USA Network">Originally aired on the USA Network</a><

In [51]:
spinner.text = f'Processing chunks: {chunks_done}/{chunks_left}'

'''Process the chunks'''
while len(chunks) > 0:
    current_link = chunks.pop()
    soup = get_soup(url + current_link)
    current_table = soup.find('table', 
    {'class': ['allpageslist', 'mw-allpages-table-chunk']})
    if 'allpageslist' in current_table.get('class'):
        child_chunks = get_links(current_table)
        chunks = chunks.union(child_chunks)
        chunks_left += len(child_chunks)
    if 'mw-allpages-table-chunk' in current_table.get('class'):
        for child in current_table.children:
            if isinstance(child, NavigableString):
                continue
            else:
                for link in child.find_all('a', href=True):
                    article_link = link['href']
                    print(article_link)
                    article_contents = get_soup(url + article).find(class="WikiaMainContentContainer")
                    article = article_link.split('/')[-1]
                    articles.add((article, article.replace('_', ' ')))
      
    chunks_done += 1
    spinner.text = f'Processing chunks: {chunks_done}/{chunks_left}'

spinner.succeed(text='Processing chunks ===> Done')

_the_Cuban_Parents
/wiki/Meet_the_Santas
/wiki/Meg_Griffin
/wiki/Mel%27s_Christmas_Carol
/wiki/Mel,_the_Magi
/wiki/Mel_Blanc
/wiki/Melchoir
/wiki/Mele_Kalikimaka
/wiki/Mele_Kalikimaka_To_You
/wiki/Melichior
/wiki/Melissa
/wiki/Melissa_Chase
/wiki/Melissa_Joan_Hart
/wiki/Melman
/wiki/Memories_of_Christmas
/wiki/Mentalo_Case
/wiki/Meowth
/wiki/Mercedes_Jones
/wiki/Merlyn
/wiki/Merry-In-Laws
/wiki/Merry_%26_Bright
/wiki/Merry_Catnip
/wiki/Merry_Chris-Hanukkah-mas
/wiki/Merry_Christmas
/wiki/Merry_Christmas!_(Hamtaro)
/wiki/Merry_Christmas,_Baby
/wiki/Merry_Christmas,_Bogg
/wiki/Merry_Christmas,_Charley
/wiki/Merry_Christmas,_Charlie_Manson!
/wiki/Merry_Christmas,_Drake_%26_Josh
/wiki/Merry_Christmas,_Drake_%26_Josh!
/wiki/Merry_Christmas,_Drake_and_Josh
/wiki/Merry_Christmas,_Father_Christmas
/wiki/Merry_Christmas,_Happy_Holidays
/wiki/Merry_Christmas,_Kenan
/wiki/Merry_Christmas,_Little_Bill
/wiki/Merry_Christmas,_Merry_Christmas
/wiki/Merry_Christmas,_Mr._Bean
/wiki/Merry_Christmas,_Mr.

In [49]:
len(articles)

8684

In [44]:
spinner = Halo(text=f'Printing {len(articles)} articles')
spinner.start()

for article in sorted(articles):
    with open(f'{article}.txt', 'w') as outfile:
        outfile.write(article)
        outfile.write('\n')

spinner.succeed(text=f'Printing {len(articles)} articles. ===> Done')

⠋ Printing 8684 articles⠙ Printing 8684 articles

OSError: [Errno 63] File name too long: '%D0%A0%D0%B0%D1%81%D1%81%D1%8B%D0%BB%D0%BA%D0%B0 %D1%80%D0%B5%D0%BA%D0%BB%D0%B0%D0%BC%D1%8B %D0%BD%D0%B0 %D1%84%D0%BE%D1%80%D1%83%D0%BC%D1%8B, %D0%BA%D0%B0%D1%82%D0%B0%D0%BB%D0%BE%D0%B3%D0%B8, %D0%B4%D0%BE%D1%81%D0%BA%D0%B8, %D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B5 %D1%80%D0%B5%D1%81%D1%83%D1%80%D1%81%D1%8B.txt'